# Different Resolutions

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import os

from torch.utils.data import DataLoader
from data.image_dataset import ImageDataset
from torchvision.transforms import v2
from torchvision.models import inception_v3
from models.convnet import ConvNet
from models.nvgaze import NVGaze
from models.incep_convnet import IncepConvNet
from training.train import *
from training.eval import *
from data.convert_labels import *
from utils.dataset_utils import check_disjoint
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
for input_dims in [(1, 127, 127), (1, 120, 160), (1, 240, 320), (1, 255, 255)]:
  subject = 1
  stride = 20
  note = ''
  mode = 'reg'
  batch_size = 64

  data_subset = f'{subject:>02}_{stride:>02}_{input_dims[2]}_{input_dims[1]}{f'_{note}' if note else ''}'
  train_dir_path = f'data/real/{data_subset}/train'
  train_annotations_file_path = f'{train_dir_path}/{data_subset}_train.csv'
  val_dir_path = f'data/real/{data_subset}/val'
  val_annotations_file_path = f'{val_dir_path}/{data_subset}_val.csv'
  test_dir_path = f'data/real/{data_subset}/test'
  test_annotations_file_path = f'{test_dir_path}/{data_subset}_test.csv'

  target_transform = None

  train_dataset = ImageDataset(
    annotations_file=train_annotations_file_path,
    img_dir=train_dir_path,
    transform=None,
    target_transform=target_transform
  )
  val_dataset = ImageDataset(
    annotations_file=val_annotations_file_path,
    img_dir=val_dir_path,
    transform=None,
    target_transform=target_transform
  )
  test_dataset = ImageDataset(
    annotations_file=test_annotations_file_path,
    img_dir=test_dir_path,
    transform=None,
    target_transform=target_transform
  )

  # assert(check_disjoint(val_dataset, test_dataset))

  # Create Dataloader
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

  out_num_features = 2

  model = NVGaze(input_dims, out_num_features, dropout_param=0.05)

  criterion = nn.MSELoss() if mode == 'reg' else nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=1e-3)
  scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 180], gamma=0.01)

  # Set-up GPU device (if available) and move model
  device = torch.device('mps' if torch.mps.is_available else 'cpu')
  model = model.to(device)
  print(f'Using device: {device}')

  results = train(model, device, mode, criterion, optimizer, train_loader, val_loader, epochs=200)

  train_error = evaluate_error(model, device, train_loader)
  val_error = evaluate_error(model, device, val_loader)
  test_error = evaluate_error(model, device, test_loader)
  train_error_str = f'Train err: {train_error.item():.6f} ({np.rad2deg(train_error.item()):.2f}°)'
  val_error_str = f'Val err:   {val_error.item():.6f} ({np.rad2deg(val_error.item()):.2f}°)'
  test_error_str = f'Test err:  {test_error.item():.6f} ({np.rad2deg(test_error.item()):.2f}°)'
  error_summary = [train_error_str, val_error_str, test_error_str]
  error_summary_str = '\n'.join(error_summary)
  print(f'({input_dims[2]}, {input_dims[1]})')
  print(error_summary_str)
  print()

Using device: mps
20
Train err: 0.032087 (1.84°)
Val err:   0.069685 (3.99°)
Test err:  0.063502 (3.64°)

Using device: mps
20
Train err: 0.032788 (1.88°)
Val err:   0.066319 (3.80°)
Test err:  0.103311 (5.92°)

Using device: mps
20
Train err: 0.032533 (1.86°)
Val err:   0.069955 (4.01°)
Test err:  0.060745 (3.48°)

Using device: mps
20
Train err: 0.039228 (2.25°)
Val err:   0.070338 (4.03°)
Test err:  0.068127 (3.90°)



In [ ]:
val_history = [4.23, 4.63, 3.99, 3.80, 4.01, 4.03]
test_history = [5.49, 5.55, 3.64, 5.92, 3.48, 3.90]